In [76]:
import pandas as pd
import numpy as np
import re
import glob
import gensim 
import itertools

from gensim import corpora, models, similarities 
from nltk.corpus import stopwords
from collections import defaultdict

In [77]:
dirs = glob.glob("/Users/krista/Desktop/w266-project-master/data/data_Houston/*.csv")
sentences = []
for dir_ in dirs:
    try:
        df = pd.read_csv(dir_, delimiter=";")
#         df = df[~df.text.str.contains(pattern)]
        new_sentences = list(df['text'].values)
        for sentence in new_sentences:
#             regex = re.compile('[^a-zA-Z]')
#             sentence = regex.sub(sentence, regex)
            sentence = re.sub(r'^https?:\/\/.*[\r\n]*', '', sentence, flags=re.MULTILINE)
            sentence = re.sub(r'^http?:\/\/.*[\r\n]*', '', sentence, flags=re.MULTILINE)
            sentence = re.sub('http[s]?://(www. )?(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', sentence)
            sentence = re.sub(" \d+", '', sentence)
            sentence = re.sub(r'\w*\d\w*', '', sentence)
            sentence = sentence.lower()
        
            sentences.append(sentence)
    except Exception as e:
        print e

expected string or buffer
Error tokenizing data. C error: Expected 10 fields in line 70, saw 11

expected string or buffer
expected string or buffer
expected string or buffer
expected string or buffer
expected string or buffer
expected string or buffer
expected string or buffer
Error tokenizing data. C error: Expected 10 fields in line 489, saw 11

expected string or buffer
expected string or buffer
expected string or buffer
expected string or buffer
expected string or buffer


In [78]:
print len(sentences)

13760


In [79]:
sentences[:10]

[' d, houston, tx, $, beds,. baths http:// bubly.us/zrdb',
 'accident cleared in #westside on i- katy fwy outbound after bingle rd, stop and go traffic back to wirt, delay of mins #traffic',
 "in the beginning i thought i was doing something now i'm straight beast mode for now @northeast\xe2\x80\xa6 ",
 'woman killed after striking-wheeler in humble http:// bubly.us/zrfw',
 ' bosphorus st, houston, tx, $, beds, baths http:// bubly.us/zrcq',
 'just posted a photo @meyerland, houston ',
 ' touchstone st, houston, tx, $, beds, bath http:// bubly.us/zrfr',
 "'young sheldon' is a genius, but jim... http:// bubly.us/zrfq",
 "gotta try torchy's tacos for #tacotuesday #roadlife #houston #texmex @torchy's tacos ",
 'catch me live at @ to bless the stage for the homie  behind the\xe2\x80\xa6 ']

In [80]:
# texts = [[word for word in sentences.lower().split() if word not in stoplist]
#           for sentence in sentences]

texts = [t for t in sentences if t not in [stopwords.words('english')] ]

texts = [t for t in texts if len(t)>4]

# # word frequency
# d=defaultdict(int)
# for lister in texts:
#     for item in lister:
#         d[item]+=1

# remove words that appear only once
# tokens=[key for key,value in d.items() if value>1]
# texts = [[word for word in document if word in tokens] for document in texts]

print texts[:10]

[' d, houston, tx, $, beds,. baths http:// bubly.us/zrdb', 'accident cleared in #westside on i- katy fwy outbound after bingle rd, stop and go traffic back to wirt, delay of mins #traffic', "in the beginning i thought i was doing something now i'm straight beast mode for now @northeast\xe2\x80\xa6 ", 'woman killed after striking-wheeler in humble http:// bubly.us/zrfw', ' bosphorus st, houston, tx, $, beds, baths http:// bubly.us/zrcq', 'just posted a photo @meyerland, houston ', ' touchstone st, houston, tx, $, beds, bath http:// bubly.us/zrfr', "'young sheldon' is a genius, but jim... http:// bubly.us/zrfq", "gotta try torchy's tacos for #tacotuesday #roadlife #houston #texmex @torchy's tacos ", 'catch me live at @ to bless the stage for the homie  behind the\xe2\x80\xa6 ']


In [81]:
# # remove words that appear only once
# frequency = defaultdict(int)
# for text in texts:
#      for token in text:
#         frequency[token] += 1

# texts = [[token for token in text if frequency[token] > 1]
#          for text in texts]

# print texts[:20]

In [82]:
from pprint import pprint  # pretty-printer
pprint (texts[:10])

[' d, houston, tx, $, beds,. baths http:// bubly.us/zrdb',
 'accident cleared in #westside on i- katy fwy outbound after bingle rd, stop and go traffic back to wirt, delay of mins #traffic',
 "in the beginning i thought i was doing something now i'm straight beast mode for now @northeast\xe2\x80\xa6 ",
 'woman killed after striking-wheeler in humble http:// bubly.us/zrfw',
 ' bosphorus st, houston, tx, $, beds, baths http:// bubly.us/zrcq',
 'just posted a photo @meyerland, houston ',
 ' touchstone st, houston, tx, $, beds, bath http:// bubly.us/zrfr',
 "'young sheldon' is a genius, but jim... http:// bubly.us/zrfq",
 "gotta try torchy's tacos for #tacotuesday #roadlife #houston #texmex @torchy's tacos ",
 'catch me live at @ to bless the stage for the homie  behind the\xe2\x80\xa6 ']


In [83]:
#  texts = [[token for token in text if frequency[token] > 1]
#            for text in texts]

In [84]:
texts =[texts[i:i+3] for i in range(0, len(texts), 3)]

In [85]:
dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(12437 unique tokens: [u'leg day! . . . . #legday #fitnessmotivation #fitspo #fitnessjourney #fitness #fitcouple \u2026 ', u'that is a sight for sore eyes!!! #harveysucks #houston @lake\u2026  / \u2026', u'drinking a hopadillo ipa by @karbachbrewing at @iah \u2014 http:// untp.beer/s/ #photo', u'#ro @flashgparks \u30fb\u30fb\u30fb one of these kids is doing his own thang... #offthewall #mj #mjj \u2026 ', u'physics n shit #rainbow #prisms #mothernature #trafficcam @houston,\u2026  / \u2026']...)


In [86]:
print(dictionary.token2id)

{u'leg day! . . . . #legday #fitnessmotivation #fitspo #fitnessjourney #fitness #fitcouple \u2026 ': 9446, u'that is a sight for sore eyes!!! #harveysucks #houston @lake\u2026  / \u2026': 7488, u'drinking a hopadillo ipa by @karbachbrewing at @iah \u2014 http:// untp.beer/s/ #photo': 924, u'#ro @flashgparks \u30fb\u30fb\u30fb one of these kids is doing his own thang... #offthewall #mj #mjj \u2026 ': 311, u'physics n shit #rainbow #prisms #mothernature #trafficcam @houston,\u2026  / \u2026': 4577, u'special edition grandmaster deluxe @avant garden houston ': 12359, u'new light christian center  event @brashaes kendrahodge @new light christian center church ': 5771, u'god yes pic.twitter.com/': 1349, u'just posted a video @galleria area ': 1803, u"it's that time!! #metime #manicure #pedicure #shellac #nails #relaxed @venezia nail spa ": 5292, u"i'm at sichuan noodles in houston, tx ": 9988, u'#chillin watching the astros play and a few other games while drinking a margarita (and getting\

In [87]:
raw_corpus=[dictionary.doc2bow(t) for t in texts]

In [88]:
print raw_corpus[:10]

[[(0, 1), (1, 1), (2, 1)], [(3, 1), (4, 1), (5, 1)], [(6, 1), (7, 1), (8, 1)], [(9, 1), (10, 1), (11, 1)], [(12, 1), (13, 1), (14, 1)], [(15, 1), (16, 1), (17, 1)], [(3, 1), (18, 1), (19, 1)], [(20, 1), (21, 1), (22, 1)], [(23, 1), (24, 1), (25, 1)], [(3, 2), (26, 1)]]


In [53]:
tfidf = gensim.models.tfidfmodel.TfidfModel(corpus=raw_corpus)

In [55]:
index = similarities.SparseMatrixSimilarity(tfidf[raw_corpus], num_features=12)

In [60]:
corpus_tfidf = tfidf[raw_corpus]

In [91]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=50)
corpus_lsi = lsi[corpus_tfidf]

In [92]:
lsi.print_topics(10)

[(0,
  u'-0.654*"the texas fashion bloggers you need to start following http:// bubly.us/" + -0.527*"top austin spots you must absolutely visit http:// bubly.us/" + -0.515*" fresh styles that will make your ponytail swing in the houston heat http:// bubly.us/" + -0.074*"friendswood set to face non-district schedule http:// bubly.us/" + -0.049*" garrett pine ln, houston, tx, $ http:// bubly.us/" + -0.036*"#job looking for #drivers in houston! carpool people to fun local events and earn some $$$! signup http:// .com/ _ pic.twitter.com/dsczhcvvha" + -0.036*" marilee ln, houston, tx, $, beds,. baths http:// bubly.us/" + -0.036*"silly faces . love you girls. therealjanai #silly #cousins #sisterlove #children #primas \u2026 " + -0.035*" nashua st, houston, tx, $, beds, bath http:// bubly.us/" + -0.035*" n everton st, houston, tx, $, http:// bubly.us/"'),
 (1,
  u'0.743*"what everyone should know before taking a probiotic http:// bubly.us/" + 0.624*" helpful tips for a stress-free road trip h